# Exercise - Neural Network

The data set for this exercise is from the banking industry. It contains data about the home loans of 2,500 bank clients. Each row represents a single loan. The columns include the characteristics of the client who used a loan. This is a binary classification task: predict whether a loan will be bad or not (1=Yes, 0=No). This is an important task for banks to prevent bad loans from being issued.

## Description of Variables

The description of variables are provided in "Loan - Data Dictionary.docx"

## Goal

Use the **loan.csv** data set and build a model to predict **BAD**. 



# Read and Prepare the Data

In [88]:
# Common imports

import pandas as pd
import numpy as np

np.random.seed(42)

# Get the data

In [89]:
#We will predict the "price" value in the data set:

loan = pd.read_csv("loan2.csv")
loan.head()

,BAD,LOAN,MORTDUE,VALUE,REASON,JOB,YOJ,DEROG,DELINQ,CLAGE,NINQ,CLNO,DEBTINC
0,0,25900,61064.0,94714.0,DebtCon,Office,2.0,0.0,0.0,98.809375,0.0,23.0,34.565944
1,0,26100,113266.0,182082.0,DebtCon,Sales,18.0,0.0,0.0,304.852469,1.0,31.0,33.193949
2,1,50000,220528.0,300900.0,HomeImp,Self,5.0,0.0,0.0,0.000000,0.0,2.0,NaN
3,1,22400,51470.0,68139.0,DebtCon,Mgr,9.0,0.0,0.0,31.168696,2.0,8.0,37.952180
4,0,20900,62615.0,87904.0,DebtCon,Office,5.0,NaN,NaN,177.864849,NaN,15.0,36.831076


# Split data (train/test)

In [90]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(loan, test_size=0.3)

# Data Prep

Perform your data prep here. You can use pipelines like we do in the tutorials. Otherwise, feel free to use your own data prep steps. Eventually, you should do the following at a minimum:<br>
- Separate inputs from target<br>
- Impute/remove missing values<br>
- Standardize the continuous variables<br>
- One-hot encode categorical variables<br>

In [91]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import FunctionTransformer

## Separate the target variable 

In [92]:
train.isna().sum()

BAD          0
LOAN         0
MORTDUE    151
VALUE       61
REASON      89
JOB         74
YOJ        156
DEROG      194
DELINQ     165
CLAGE      105
NINQ       153
CLNO        76
DEBTINC    556
dtype: int64

In [93]:
train_target = train['BAD']
test_target = test['BAD']

train_inputs = train.drop(['BAD'], axis=1)
test_inputs = test.drop(['BAD'], axis=1)

## Feature Engineering: Derive a new column

Examples:
- Ratio of delinquent to total number of credit lines
- Ratio of loan to value of current property
- Convert yr_renovated to a binary variable (i.e., renovated or not)
- (etc.)

In [94]:
def new_col(df):
    #Create a copy so that we don't overwrite the existing dataframe
    df1 = df.copy()
    
    df1['loan_ratio'] = (df1['LOAN'] / df1['VALUE']).fillna(0)
    
    return df1[['loan_ratio']]

In [95]:
new_col(train_inputs)

,loan_ratio
1552,0.289149
2290,0.175701
1398,0.051985
1775,0.216491
2299,0.165515
...,...
1638,0.081112
1095,0.198824
1130,0.297348
1294,0.072863


In [96]:
train_inputs

,LOAN,MORTDUE,VALUE,REASON,JOB,YOJ,DEROG,DELINQ,CLAGE,NINQ,CLNO,DEBTINC
1552,14600,15411.0,50493.0,HomeImp,Office,24.0,0.0,0.0,259.746891,0.0,19.0,30.568713
2290,23300,103783.0,132612.0,DebtCon,Sales,1.0,0.0,NaN,120.347967,0.0,19.0,203.312149
1398,5700,80437.0,109647.0,HomeImp,ProfExe,9.5,0.0,0.0,104.400000,0.0,14.0,NaN
1775,27300,97373.0,126102.0,DebtCon,Office,10.0,0.0,1.0,139.132598,0.0,28.0,39.173554
2299,9700,53527.0,58605.0,DebtCon,Office,4.0,0.0,0.0,75.584543,3.0,12.0,36.709539
...,...,...,...,...,...,...,...,...,...,...,...,...
1638,7000,66000.0,86300.0,DebtCon,Other,3.0,1.0,1.0,242.433333,0.0,8.0,NaN
1095,16900,63800.0,85000.0,DebtCon,Other,2.0,0.0,6.0,127.800000,2.0,33.0,NaN
1130,15700,35873.0,52800.0,HomeImp,Other,10.0,0.0,0.0,128.235914,0.0,10.0,20.950252
1294,12900,149297.0,177044.0,HomeImp,Office,4.0,0.0,0.0,205.873921,0.0,56.0,34.975734


##  Identify the numeric, binary, and categorical columns

In [97]:
# Identify the numerical columns
numeric_columns = train_inputs.select_dtypes(include=[np.number]).columns.to_list()

# Identify the categorical columns
categorical_columns = train_inputs.select_dtypes('object').columns.to_list()

In [98]:
numeric_columns

['LOAN',
 'MORTDUE',
 'VALUE',
 'YOJ',
 'DEROG',
 'DELINQ',
 'CLAGE',
 'NINQ',
 'CLNO',
 'DEBTINC']

In [99]:
categorical_columns

['REASON', 'JOB']

In [100]:
# define your feature engineered column(s) here
feat_eng_columns = ['LOAN', 'VALUE']

# Pipeline

In [101]:
numeric_transformer = Pipeline(steps=[
                ('imputer', SimpleImputer(strategy='median')),
                ('scaler', StandardScaler())])

In [102]:
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='unknown')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

In [103]:
# Create a pipeline for the transformed column here
my_new_column = Pipeline(steps=[('my_new_column', FunctionTransformer(new_col)),                            
                               ('scaler', StandardScaler())])

In [104]:
preprocessor = ColumnTransformer([
        ('num', numeric_transformer, numeric_columns),
        ('cat', categorical_transformer, categorical_columns),
        ('trans', my_new_column, feat_eng_columns)],   
        remainder='drop')

#passtrough is an optional step. You don't have to use it.

# Transform: fit_transform() for TRAIN

In [105]:
#Fit and transform the train data
train_x = preprocessor.fit_transform(train_inputs)

train_x

array([[-0.31412013, -1.30301181, -0.86148829, ...,  0.        ,
         0.        ,  0.45952724],
       [ 0.45733454,  0.7398414 ,  0.58636192, ...,  0.        ,
         0.        , -0.1947115 ],
       [-1.10330939,  0.2001631 ,  0.18146318, ...,  0.        ,
         0.        , -0.90815883],
       ...,
       [-0.21657988, -0.83000156, -0.82081329, ...,  0.        ,
         0.        ,  0.50681238],
       [-0.46486414,  1.79196675,  1.36974799, ...,  0.        ,
         0.        , -0.78775744],
       [-0.31412013, -0.08740643, -0.21782887, ...,  0.        ,
         0.        , -0.24017983]])

In [106]:
train_x.shape

(1750, 21)

# Tranform: transform() for TEST

In [107]:
# Transform the test data
test_x = preprocessor.transform(test_inputs)

test_x

array([[ 0.06717356,  0.36706438,  0.32127798, ...,  0.        ,
         0.        , -0.28095348],
       [ 0.32432512,  0.57631513,  0.42769944, ...,  1.        ,
         0.        , -0.19092663],
       [-0.33185472,  0.41209537,  0.12227549, ...,  0.        ,
         0.        , -0.42666598],
       ...,
       [-0.8993616 , -0.51071616, -0.32326299, ...,  0.        ,
         0.        , -0.63852433],
       [-0.5446698 , -0.82706576, -0.8366813 , ...,  0.        ,
         0.        ,  0.12542485],
       [-0.5446698 , -0.06422056, -0.11380525, ...,  0.        ,
         0.        , -0.4630389 ]])

In [108]:
test_x.shape

(750, 21)

# Calculate the Baseline

In [109]:
from sklearn.dummy import DummyClassifier

dummy_clf = DummyClassifier(strategy="most_frequent")

dummy_clf.fit(train_x, train_target)

DummyClassifier(strategy='most_frequent')

In [110]:
from sklearn.metrics import accuracy_score

In [111]:
#Baseline Train Accuracy
dummy_train_pred = dummy_clf.predict(train_x)

baseline_train_acc = accuracy_score(train_target, dummy_train_pred)

print('Baseline Train Accuracy: {}' .format(baseline_train_acc))

Baseline Train Accuracy: 0.6034285714285714


In [112]:
#Baseline Test Accuracy
dummy_test_pred = dummy_clf.predict(test_x)

baseline_test_acc = accuracy_score(test_target, dummy_test_pred)

print('Baseline Test Accuracy: {}' .format(baseline_test_acc))

Baseline Test Accuracy: 0.5773333333333334


# Train a shallow (one-layer) NN model

In [113]:
from sklearn.neural_network import MLPClassifier

#Default settings create 1 hidden layer with 100 neurons
mlp_clf = MLPClassifier(hidden_layer_sizes=(100,))

mlp_clf.fit(train_x, train_target)

C:\Users\17272\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:684: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


MLPClassifier()

### Calculate the accuracy

In [114]:
from sklearn.metrics import accuracy_score

#Predict the train values
train_target_pred = mlp_clf.predict(train_x)

#Train accuracy
accuracy_score(train_target, train_target_pred)

0.9125714285714286

In [116]:
#Predict the test values
test_target_pred = mlp_clf.predict(test_x)

#Test accuracy
accuracy_score(test_target, test_target_pred)



0.8346666666666667

# Train a deep (multi-layered) NN model 

In [117]:
dnn_clf = MLPClassifier(hidden_layer_sizes=(100,80,60,40,20,10),
                       max_iter=1000)

dnn_clf.fit(train_x, train_target)

MLPClassifier(hidden_layer_sizes=(100, 80, 60, 40, 20, 10), max_iter=1000)

In [118]:

#Let's check the number of iterations:
dnn_clf.n_iter_


88

In [119]:
#Let's check the number of layers:
dnn_clf.n_layers_

8

### Calculate the accuracy

In [120]:
#Predict the train values
train__pred = dnn_clf.predict(train_x)

#Train accuracy
accuracy_score(train_target, train_target_pred)

0.9125714285714286

In [121]:
#Predict the test values
test_target_pred = dnn_clf.predict(test_x)

#Test accuracy
accuracy_score(test_target, test_target_pred)

0.8746666666666667

# Optional: try grid search

In [122]:
mlp_gs = MLPClassifier(max_iter=100)
parameter_space = {
    'hidden_layer_sizes': [(10,30,10),(20,)],
    'activation': ['tanh', 'relu'],
    'solver': ['sgd', 'adam'],
    'alpha': [0.0001, 0.05],
    'learning_rate': ['constant','adaptive'],
}
from sklearn.model_selection import GridSearchCV
clf = GridSearchCV(mlp_gs, parameter_space, n_jobs=-1, cv=5)
clf.fit(train_x, train_target)

C:\Users\17272\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:684: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(


GridSearchCV(cv=5, estimator=MLPClassifier(max_iter=100), n_jobs=-1,
             param_grid={'activation': ['tanh', 'relu'],
                         'alpha': [0.0001, 0.05],
                         'hidden_layer_sizes': [(10, 30, 10), (20,)],
                         'learning_rate': ['constant', 'adaptive'],
                         'solver': ['sgd', 'adam']})

In [123]:
print('Best parameters found:\n', clf.best_params_)

Best parameters found:
 {'activation': 'relu', 'alpha': 0.0001, 'hidden_layer_sizes': (10, 30, 10), 'learning_rate': 'adaptive', 'solver': 'adam'}


In [124]:
#Train accuracy:
train_target_pred = clf.best_estimator_.predict(train_x)

print(accuracy_score(train_target, train_target_pred))

0.8531428571428571


In [125]:
#Test accuracy:
test_target_pred = clf.best_estimator_.predict(test_x)

print(accuracy_score(test_target, test_target_pred))

0.7933333333333333
